In [1]:
import toolkit

In [2]:
def caseStudyAnalysis(dataSet):
    print dataSet['times']
    print dataSet['data'].shape
    print dataSet['data']['entity'].nunique()

    from sklearn.tree import DecisionTreeClassifier
    modelInstance = DecisionTreeClassifier(max_leaf_nodes=8, criterion='entropy')
    modelSimpler = DecisionTreeClassifier(max_leaf_nodes=4, criterion='entropy')
    churnModel = toolkit.refinement.makeAndUpdateModel(rootDirectory, dataSet['data'], 2, 'netchurn', modelInstance, modelSimpler, visualize=False, scoreOnly=False) 

    folds = 5
    churnModelMoreFolds = toolkit.refinement.makeAndUpdateModel(rootDirectory, dataSet['data'], folds, 'netchurn', modelInstance, modelSimpler, scoreOnly=False) 

    print dataSet['data']['netchurn'].mean()
    print dataSet['data']['netchurn'].var()
    print dataSet['data']['netchurn'].std()
    print dataSet['data']['netchurn'].max()
    print dataSet['data']['netchurn'].min()

    churnBinnedCategories = ['churnLow','churnMedium','churnHigh','churnHigher', 'churnHighest']
    dataSetUpdated = toolkit.utilities.addBinnedResponseCategory(dataSet['data'], 'netchurn', churnBinnedCategories)

    from sklearn.tree import DecisionTreeClassifier
    modelInstance = DecisionTreeClassifier(max_leaf_nodes=8, criterion='entropy')
    churnModelCategories = toolkit.refinement.makeAndUpdateModel(rootDirectory, dataSetUpdated, 2, churnBinnedCategories, modelInstance, modelSimpler, visualize=False, scoreOnly=False) 

    folds = 3
    churnModelCategories = toolkit.refinement.makeAndUpdateModel(rootDirectory, dataSetUpdated, folds, churnBinnedCategories, modelInstance, modelSimpler, visualize=False, scoreOnly=False) 

    addedModel = toolkit.refinement.makeAndUpdateModel(rootDirectory, dataSet['data'], 2, 'added', modelInstance, modelSimpler, visualize=False, scoreOnly=False) 

    alteredData = dataSet['data'].drop(['netchurn','deleted'], axis=1)
    addedModel = toolkit.refinement.makeAndUpdateModel(rootDirectory, alteredData, 2, 'added', modelInstance, modelSimpler, visualize=False, scoreOnly=False) 

    alteredData2 = dataSet['data'].drop(['netchurn','deleted','n-revs'], axis=1)
    addedModel2 = toolkit.refinement.makeAndUpdateModel(rootDirectory, alteredData2, 2, 'added', modelInstance, modelSimpler, visualize=False, scoreOnly=False) 

    alteredData3 = dataSet['data'].drop(['netchurn','deleted','n-revs','n-authors'], axis=1)
    addedModel3 = toolkit.refinement.makeAndUpdateModel(rootDirectory, alteredData3, 2, 'added', modelInstance, modelSimpler, visualize=False, scoreOnly=False) 

    from sklearn.tree import DecisionTreeRegressor
    modelInstanceR = DecisionTreeRegressor(max_leaf_nodes=8)
    modelInstanceRsimpler = DecisionTreeRegressor(max_leaf_nodes=4)    
    alteredData4 = dataSet['data'].drop(['netchurn','deleted','n-revs','n-authors','fractal-value'], axis=1)
    nlineModelR = toolkit.refinement.makeAndUpdateModel(rootDirectory, alteredData4, 2, 'nline', modelInstanceR, modelInstanceRsimpler, visualize=False, scoreOnly=False)

    dataSet['data']['nline'].corr(dataSet['data']['indent_lines'], method='spearman')

    alteredData5 = dataSet['data'].drop(['indent_lines','nchar','nstatement','nidentifier'], axis=1)
    nlineModelR2 = toolkit.refinement.makeAndUpdateModel(rootDirectory, alteredData5, 2, 'nline', modelInstanceR, modelInstanceRsimpler, visualize=False, scoreOnly=False)

    alteredData6 = dataSet['data'].drop(['indent_lines','nchar','nstatement','nidentifier', 'nfunction','nfunction2','nfunction3'], axis=1)
    nlineModelR3 = toolkit.refinement.makeAndUpdateModel(rootDirectory, alteredData6, 2, 'nline', modelInstanceR, modelInstanceRsimpler, visualize=False, scoreOnly=False)

    alteredData7 = dataSet['data'].drop(['indent_lines','nchar','nstatement','nidentifier', 'nfunction','nfunction2','nfunction3', 'unique_nidentifier'], axis=1)
    nlineModelR4 = toolkit.refinement.makeAndUpdateModel(rootDirectory, alteredData7, 2, 'nline', modelInstanceR, modelInstanceRsimpler, visualize=False, scoreOnly=False)

    cycloData = dataSet['data'].drop(['cyclomatic_sd', 'cyclomatic_mean'], axis=1)
    cycloModelR = toolkit.refinement.makeAndUpdateModel(rootDirectory, cycloData, 2, 'cyclomatic_max', modelInstanceR, visualize=False, scoreOnly=False)

    cycloData2 = dataSet['data'].drop(['halstead_sd','nidentifier','halstead_mean','halstead_min','cyclomatic_sd', 'cyclomatic_mean', 'halstead_max','nstatement','statement_nesting_mean'], axis=1)
    cycloModelR = toolkit.refinement.makeAndUpdateModel(rootDirectory, cycloData2, 2, 'cyclomatic_max', modelInstanceR, modelInstanceRsimpler, visualize=False, scoreOnly=False)

    indentData = dataSet['data'].drop(['indent_sd','indent_median','indent_max','indent_lines'],axis=1)
    indentModelR = toolkit.refinement.makeAndUpdateModel(rootDirectory, indentData, 2, 'indent_mean', modelInstanceR, modelInstanceRsimpler, visualize=False, scoreOnly=False)

    indentModelC = toolkit.refinement.makeAndUpdateModel(rootDirectory, indentData, 2, 'indent_mean', modelInstance, modelSimpler, visualize=False, scoreOnly=False)

    indentBinnedCategories = ['iLow','iMedium','iHigh','iVeryHigh']
    dataSetUpdated = toolkit.utilities.addBinnedResponseCategory(indentData, 'indent_mean', indentBinnedCategories)
    indentModelC = toolkit.refinement.makeAndUpdateModel(rootDirectory, dataSetUpdated, 2, indentBinnedCategories, modelInstance, modelSimpler, visualize=False, scoreOnly=False)

    import psutil
    cores = psutil.cpu_count()

    from sklearn.ensemble import RandomForestClassifier
    modelCF = RandomForestClassifier(n_estimators=500, criterion='entropy', n_jobs=cores)
    modelCFsimpler = RandomForestClassifier(n_estimators=50, max_leaf_nodes=4, criterion='entropy', n_jobs=cores)    
    updatedModelCF = toolkit.refinement.makeAndUpdateModel(rootDirectory, dataSetUpdated, 2, indentBinnedCategories, modelCF, modelCFsimpler, scoreOnly=False, visualize=False)

In [3]:
rootDirectory = '../dataSets/vimStudy/'
metricsDataVim = toolkit.data.gatherTimeMetrics(rootDirectory, 'https://github.com/vim/vim', rootDirectory+'vim/', '*/*.c */*.h *.c *.h', ['indent','c'], skipEvery=50)
caseStudyAnalysis(metricsDataVim)

105
(2684, 64)
193
Response variable was netchurn
Model.score: 0.152125
accuracy_score: 0.152125
             precision    recall  f1-score   support

      -3522       0.00      0.00      0.00         1
      -1407       0.00      0.00      0.00         1
      -1279       0.00      0.00      0.00         1
       -784       0.00      0.00      0.00         1
       -554       0.00      0.00      0.00         1
       -534       0.00      0.00      0.00         1
       -518       0.00      0.00      0.00         1
       -474       0.00      0.00      0.00         1
       -258       0.00      0.00      0.00         1
       -173       0.00      0.00      0.00         1
       -159       0.00      0.00      0.00         1
       -152       0.00      0.00      0.00         1
        -95       0.00      0.00      0.00         1
        -68       0.00      0.00      0.00         1
        -62       0.00      0.00      0.00         1
        -40       0.00      0.00      0.00         1
 

/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/utils/__init__.py:93: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


Model.score: 0.990654
accuracy_score: 0.990654
             precision    recall  f1-score   support

          0       1.00      0.99      0.99       385
          1       0.97      0.99      0.98       150

avg / total       0.99      0.99      0.99       535

roc_auc_score: 0.991472
      name  importance
3    added    0.900649
4  deleted    0.099351
Model.score: 0.985047
accuracy_score: 0.985047
             precision    recall  f1-score   support

          0       0.99      1.00      0.99       498
          1       0.94      0.84      0.89        37

avg / total       0.98      0.99      0.98       535

roc_auc_score: 0.916911
      name  importance
0    added    0.906347
1  deleted    0.093653
Model.score: 0.981308
accuracy_score: 0.981308
             precision    recall  f1-score   support

          0       0.98      1.00      0.99       513
          1       0.93      0.59      0.72        22

avg / total       0.98      0.98      0.98       535

roc_auc_score: 0.794480
    

                name  importance
33     ncomment_char    0.854610
8   line_length_mean    0.079664
20             ngoto    0.026522
45   cyclomatic_mean    0.013761
17         ninternal    0.013422
51         indent_sd    0.009573
7           netchurn    0.002446
Model.score: 0.831832
               name  importance
3     ncomment_char    0.889481
0  line_length_mean    0.082914
2             ngoto    0.027605
Response variable was nline
Model.score: 0.871940
                        name  importance
33             ncomment_char    0.868372
8           line_length_mean    0.079664
20                     ngoto    0.026522
145  entity_src_if_py_both.h    0.013422
50                 indent_sd    0.009573
26                   nstruct    0.002446
Model.score: 0.831832
               name  importance
2     ncomment_char    0.889481
0  line_length_mean    0.082914
1             ngoto    0.027605
Response variable was cyclomatic_max
Model.score: 0.855582
                     name  importance
47

/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Model.score: 0.966443
accuracy_score: 0.966443
             precision    recall  f1-score   support

          0       0.99      0.96      0.98       409
          1       0.97      0.98      0.97       827
          2       0.92      0.88      0.90       105
          3       0.00      0.00      0.00         0

avg / total       0.97      0.97      0.97      1341

roc_auc_score cannot be computed for this test set
                         name    importance
17     statement_nesting_mean  4.726575e-02
55                nidentifier  4.706314e-02
15                 nstatement  4.581494e-02
21                  ninternal  4.565172e-02
51            cyclomatic_mean  4.299874e-02
20       statement_nesting_sd  4.008556e-02
33                      nvoid  3.660164e-02
14                  nfunction  3.207892e-02
53             cyclomatic_max  3.186061e-02
9                       nline  3.085556e-02
54              cyclomatic_sd  2.971494e-02
43                 nfunction2  2.803270e-02
40       

In [4]:
rootDirectory = '../dataSets/openSSLstudy/'
metricsDataOpenSSL = toolkit.data.gatherTimeMetrics(rootDirectory, 'https://github.com/openssl/openssl', rootDirectory+'openssl/', '*/*.c */*.h *.c *.h', ['indent','c'], skipEvery=50)
caseStudyAnalysis(metricsDataOpenSSL)

180
(14042, 64)
1351
Response variable was netchurn
Model.score: 0.373504
accuracy_score: 0.373504
             precision    recall  f1-score   support

      -1357       0.00      0.00      0.00         1
      -1342       0.00      0.00      0.00         1
      -1119       0.00      0.00      0.00         1
      -1059       0.00      0.00      0.00         1
      -1040       0.00      0.00      0.00         1
       -972       0.00      0.00      0.00         1
       -967       0.00      0.00      0.00         1
       -924       0.00      0.00      0.00         1
       -896       0.00      0.00      0.00         1
       -825       0.00      0.00      0.00         1
       -725       0.00      0.00      0.00         1
       -709       0.00      0.00      0.00         1
       -592       0.00      0.00      0.00         1
       -567       0.00      0.00      0.00         1
       -564       0.00      0.00      0.00         1
       -544       0.00      0.00      0.00         1

roc_auc_score cannot be computed for this test set
      name  importance
3    added    0.560976
4  deleted    0.439024
Model.score: 0.368376
accuracy_score: 0.368376
             precision    recall  f1-score   support

      -1357       0.00      0.00      0.00         1
      -1342       0.00      0.00      0.00         1
      -1119       0.00      0.00      0.00         1
      -1059       0.00      0.00      0.00         1
      -1040       0.00      0.00      0.00         1
       -972       0.00      0.00      0.00         1
       -967       0.00      0.00      0.00         1
       -924       0.00      0.00      0.00         1
       -896       0.00      0.00      0.00         1
       -825       0.00      0.00      0.00         1
       -725       0.00      0.00      0.00         1
       -709       0.00      0.00      0.00         1
       -592       0.00      0.00      0.00         1
       -567       0.00      0.00      0.00         1
       -564       0.00      0.00     

Model.score: 0.928037
accuracy_score: 0.928037
             precision    recall  f1-score   support

          0       0.94      0.99      0.96      2546
          1       0.73      0.36      0.48       261

avg / total       0.92      0.93      0.92      2807

roc_auc_score: 0.671484
      name  importance
3    added    0.829416
4  deleted    0.170584
Model.score: 0.916281
accuracy_score: 0.916281
             precision    recall  f1-score   support

          0       0.91      0.98      0.94      2037
          1       0.94      0.74      0.83       770

avg / total       0.92      0.92      0.91      2807

roc_auc_score: 0.862750
      name  importance
3    added    0.688105
4  deleted    0.206204
6      soc    0.072749
7    nchar    0.032942
Model.score: 0.941931
accuracy_score: 0.941931
             precision    recall  f1-score   support

          0       0.98      0.95      0.96      2168
          1       0.84      0.92      0.88       639

avg / total       0.95      0.94    

Model.score: 0.877208
accuracy_score: 0.877208
             precision    recall  f1-score   support

          0       0.92      0.94      0.93      6070
          1       0.56      0.45      0.50       950

avg / total       0.87      0.88      0.87      7020

roc_auc_score: 0.697257
            name  importance
3            soc    0.464573
53  indent_lines    0.290158
5          nline    0.202204
51   nidentifier    0.023132
58          time    0.019933
Model.score: 0.881766
accuracy_score: 0.881766
             precision    recall  f1-score   support

          0       0.92      0.95      0.93      6070
          1       0.58      0.45      0.51       950

avg / total       0.87      0.88      0.88      7020

roc_auc_score: 0.699892
           name  importance
0           soc    0.565268
3  indent_lines    0.222935
1         nline    0.211797
Response variable was added
Model.score: 0.877208
accuracy_score: 0.877208
             precision    recall  f1-score   support

          0  

Model.score: 0.578348
accuracy_score: 0.578348
             precision    recall  f1-score   support

          0       0.81      0.54      0.65      2653
          1       0.60      0.81      0.69      3219
          2       0.39      0.02      0.04      1102
          3       0.00      0.00      0.00        46

avg / total       0.64      0.58      0.57      7020

roc_auc_score: 0.604601
                        name  importance
48            cyclomatic_max    0.127797
23                     ngoto    0.089884
50               nidentifier    0.088911
49             cyclomatic_sd    0.074977
9                      nline    0.073246
16    statement_nesting_mean    0.072889
8                      nchar    0.067221
15                nstatement    0.066853
19      statement_nesting_sd    0.047098
13            line_length_sd    0.041183
42              halstead_max    0.040695
10          line_length_mean    0.036797
51        unique_nidentifier    0.035867
31                  ncomment    0.

In [5]:
rootDirectory = '../dataSets/httpdStudy/'
metricsDataHTTPD = toolkit.data.gatherTimeMetrics(rootDirectory, 'https://github.com/apache/httpd', rootDirectory+'httpd/', '*/*.c */*.h *.c *.h', ['indent','c'], branch='trunk', skipEvery=50)
caseStudyAnalysis(metricsDataHTTPD)

253
(10054, 64)
1866
Response variable was netchurn
Model.score: 0.384202
accuracy_score: 0.384202
             precision    recall  f1-score   support

      -3255       0.00      0.00      0.00         1
      -2551       0.00      0.00      0.00         1
      -1890       0.00      0.00      0.00         1
      -1746       0.00      0.00      0.00         1
      -1715       0.00      0.00      0.00         1
      -1527       0.00      0.00      0.00         1
      -1499       0.00      0.00      0.00         1
      -1409       0.00      0.00      0.00         1
      -1316       0.00      0.00      0.00         1
      -1194       0.00      0.00      0.00         1
      -1192       0.00      0.00      0.00         1
      -1113       0.00      0.00      0.00         1
      -1060       0.00      0.00      0.00         1
       -993       0.00      0.00      0.00         1
       -862       0.00      0.00      0.00         1
       -830       0.00      0.00      0.00         1

Model.score: 0.381815
accuracy_score: 0.381815
             precision    recall  f1-score   support

      -3255       0.00      0.00      0.00         1
      -2551       0.00      0.00      0.00         1
      -1890       0.00      0.00      0.00         1
      -1746       0.00      0.00      0.00         1
      -1715       0.00      0.00      0.00         1
      -1527       0.00      0.00      0.00         1
      -1499       0.00      0.00      0.00         1
      -1409       0.00      0.00      0.00         1
      -1316       0.00      0.00      0.00         1
      -1194       0.00      0.00      0.00         1
      -1192       0.00      0.00      0.00         1
      -1113       0.00      0.00      0.00         1
      -1060       0.00      0.00      0.00         1
       -993       0.00      0.00      0.00         1
       -862       0.00      0.00      0.00         1
       -830       0.00      0.00      0.00         1
       -738       0.00      0.00      0.00         

Model.score: 0.930811
accuracy_score: 0.930811
             precision    recall  f1-score   support

          0       0.96      0.96      0.96      1714
          1       0.75      0.79      0.77       295

avg / total       0.93      0.93      0.93      2009

roc_auc_score: 0.872453
      name  importance
3    added    0.480045
6      soc    0.328902
4  deleted    0.191053
Model.score: 0.968143
accuracy_score: 0.968143
             precision    recall  f1-score   support

          0       0.97      1.00      0.98      1850
          1       0.94      0.64      0.76       159

avg / total       0.97      0.97      0.97      2009

roc_auc_score: 0.818863
      name  importance
3    added    0.429052
6      soc    0.365083
4  deleted    0.205865
Model.score: 0.959681
accuracy_score: 0.959681
             precision    recall  f1-score   support

          0       0.98      0.97      0.98      1787
          1       0.80      0.85      0.82       222

avg / total       0.96      0.96    

Model.score: 0.911659
accuracy_score: 0.911659
             precision    recall  f1-score   support

          0       0.95      0.95      0.95      4572
          1       0.51      0.55      0.53       454

avg / total       0.91      0.91      0.91      5026

roc_auc_score: 0.747100
             name  importance
53   indent_lines    0.858392
51    nidentifier    0.050038
1       n-authors    0.029272
3             soc    0.028608
2   fractal-value    0.024985
58           time    0.008705
Model.score: 0.943494
accuracy_score: 0.943494
             precision    recall  f1-score   support

          0       0.95      0.99      0.97      4572
          1       0.81      0.49      0.61       454

avg / total       0.94      0.94      0.94      5026

roc_auc_score: 0.740791
           name  importance
4  indent_lines    0.915421
3   nidentifier    0.053362
0     n-authors    0.031217
Response variable was added
Model.score: 0.911659
accuracy_score: 0.911659
             precision    recal

Model.score: 0.466972
accuracy_score: 0.466972
             precision    recall  f1-score   support

          0       0.95      0.62      0.75      2149
          1       0.47      0.28      0.35      2345
          2       0.31      0.72      0.43       505
          3       0.00      0.00      0.00        27

avg / total       0.66      0.47      0.53      5026

roc_auc_score: 0.642459
                        name  importance
15                nstatement    0.185892
45                nfunction3    0.084853
14                 nfunction    0.081698
43              halstead_max    0.073570
47           cyclomatic_mean    0.054791
49            cyclomatic_max    0.051725
39                nfunction2    0.049576
16    statement_nesting_mean    0.047795
20                 ninternal    0.047481
41             halstead_mean    0.043362
50             cyclomatic_sd    0.041822
44               halstead_sd    0.039670
29                     nvoid    0.028184
9                      nline    0.

In [6]:
rootDirectory = '../dataSets/nginxStudy/'
metricsDataNginx = toolkit.data.gatherTimeMetrics(rootDirectory, 'https://github.com/nginx/nginx', rootDirectory+'nginx/', '*/*.c */*.h *.c *.h', ['indent','c'], skipEvery=50)
caseStudyAnalysis(metricsDataNginx)

88
(3457, 64)
481
Response variable was netchurn
Model.score: 0.116966
accuracy_score: 0.116966
             precision    recall  f1-score   support

      -1656       0.00      0.00      0.00         1
      -1653       0.00      0.00      0.00         1
      -1512       0.00      0.00      0.00         1
      -1500       0.00      0.00      0.00         1
      -1471       0.00      0.00      0.00         1
      -1345       0.00      0.00      0.00         1
      -1335       0.00      0.00      0.00         1
      -1182       0.00      0.00      0.00         1
      -1037       0.00      0.00      0.00         1
      -1031       0.00      0.00      0.00         1
       -973       0.00      0.00      0.00         1
       -810       0.00      0.00      0.00         1
       -756       0.00      0.00      0.00         1
       -628       0.00      0.00      0.00         1
       -603       0.00      0.00      0.00         1
       -587       0.00      0.00      0.00         1
  

      name  importance
3    added    0.909178
4  deleted    0.090822
Model.score: 0.933333
accuracy_score: 0.933333
             precision    recall  f1-score   support

          0       0.97      0.93      0.95       466
          1       0.87      0.93      0.90       224

avg / total       0.94      0.93      0.93       690

roc_auc_score: 0.933256
      name  importance
3    added    0.889015
4  deleted    0.110985
Model.score: 0.985507
accuracy_score: 0.985507
             precision    recall  f1-score   support

          0       1.00      0.98      0.99       635
          1       0.85      1.00      0.92        55

avg / total       0.99      0.99      0.99       690

roc_auc_score: 0.992126
      name  importance
0    added    0.953268
1  deleted    0.046732
Model.score: 0.956522
accuracy_score: 0.956522
             precision    recall  f1-score   support

          0       1.00      0.95      0.97       550
          1       0.83      0.99      0.90       140

avg / total  

                            name  importance
56                    indent_max    0.406422
3                          nchar    0.326642
9                      nfunction    0.121963
46               cyclomatic_mean    0.038840
2                            soc    0.036018
135  entity_src_core_ngx_shmtx.c    0.035149
57                          time    0.034966
Model.score: 0.702953
accuracy_score: 0.702953
             precision    recall  f1-score   support

          0       0.77      0.83      0.80      1231
          1       0.48      0.38      0.42       496

avg / total       0.69      0.70      0.69      1727

roc_auc_score: 0.606852
         name  importance
4  indent_max    0.475332
1       nchar    0.382025
2   nfunction    0.142642
Response variable was nline
Model.score: 0.930944
            name  importance
51  indent_lines    0.986345
9     nstatement    0.013655
Model.score: 0.785440
           name  importance
1  indent_lines         1.0
Response variable was nline
Model.s

Model.score: 0.745223
accuracy_score: 0.745223
             precision    recall  f1-score   support

          0       0.92      0.93      0.93       817
          1       0.75      0.82      0.78       566
          2       0.78      0.21      0.33       297
          3       0.00      0.00      0.00        47

avg / total       0.81      0.75      0.75      1727

roc_auc_score: 0.717031
                        name  importance
39                nfunction3    0.162238
13                nstatement    0.142235
14    statement_nesting_mean    0.085163
17      statement_nesting_sd    0.070620
18                 ninternal    0.054652
12                 nfunction    0.049801
33                nfunction2    0.048272
44             cyclomatic_sd    0.045382
41           cyclomatic_mean    0.040160
7                      nline    0.034415
45               nidentifier    0.032990
35             halstead_mean    0.030786
43            cyclomatic_max    0.030349
16     statement_nesting_max    0.